In [3]:
import re
import pandas as pd
from pandarallel import pandarallel

pandarallel.initialize()

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [ ]:
def strip_scientificName(row):
    sciName = row["scientificName"]

    if pd.isna(sciName):
        return row

    row["scientificName"] = sciName.strip()
    return row


def remove_parentheses(row):
    sciName = row["scientificName"]

    if pd.isna(sciName):
        return row

    sciName = re.sub(r"\([^)]+\)", "", sciName)
    sciName = re.sub(r" +", " ", sciName).strip()
    row["scientificName"] = sciName
    return row


def remove_authorship(row):
    sciName = row["scientificName"]

    if pd.isna(sciName):
        return row

    sciName = re.sub(r" [A-Z][a-z]+,\s+\d{4}\s*$", "", sciName)
    sciName = re.sub(r" +", " ", sciName).strip()
    row["scientificName"] = sciName
    return row


def do_cleaning(row):
    row = remove_parentheses(row)
    row = remove_authorship(row)
    row = strip_scientificName(row)
    return row
